In [8]:
def one_check(i, rects):
    (x,y,w,h) = rects[i]
    
    wh_ratio = h / w

    if wh_ratio > 3:
        return True

    else:
        return False

In [9]:
def decimal_check(decimal_count, i, rects, size, image, str_num):
    
    (x,y,w,h) = rects[i]
    
    ret_val = False
    
#     print(w*h,i)
    
    if w*h <= 190:
        ret_val = True
        if w*h >=30:
            if decimal_count==0:
                if i>0 and i< size-1:
                    (x1,y1,w1,h1)=rects[i-1]
                    (x2,y2,w2,h2)=rects[i+1]
                    t=x+w/2
                    d=y+h/2
        #             print(t,d)
                    if t<x2+w2/2 and t>x1+w1/2 and d>y1 and d<y1+h1:
                        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
                        cv2.putText(image,'d',(x+2,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1)
        #                 show("Recognized",image)
                        ret_val = True
                        decimal_count += 1
                        str_num += '.'

                elif i==0 and size>1:
                    (x1,y1,w1,h1)=rects[i+1]
                    t=x+w/2
                    d=y+h/2
                    if t<x1 and d>y1 and d<y1+h1:
#                         print(i,decimal_count)
                        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
                        cv2.putText(image,'d',(x+2,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1)
        #                 show("Recognized",image)
                        ret_val = True
                        decimal_count += 1
                        str_num += '.'

    #             elif i==size-1 and size>1:
    #                 (x1,y1,w1,h1)=rects[i-1]
    #                 t=x+w/2
    #                 d=y+h/2
    #                 if t>x1+w1 and d>y1 and d<y1+h1:
    #                     cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
    #                     cv2.putText(image,'d',(x+2,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1)
    #     #                 show("Recognized",image)
    #                     ret_val = True    
    #                     decimal_count += 1
        

    return str_num, decimal_count, ret_val, image

In [15]:
def pre_processing(image):
    image = imutils.resize(image,width=150)
    #image = cv2.medianBlur(image,3)
    image = cv2.GaussianBlur(image,(9,9),0)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    
#     """
#     Start of Processing
#     """
    
#     # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#     # opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    
# #     kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(6,6))
#     blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)
    
# #     show('image1',blackhat)
    
#     _,thresh = cv2.threshold(blackhat,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
# #     thresh = cv2.dilate(thresh,None)

    
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
#     thresh = cv2.erode(thresh,kernel,iterations = 1)
    
#     thresh = cv2.dilate(thresh,None)
    
#     """
#     End of Processing
#     """
    


#     thresh = thresh & kernel
    


#     show('image1',thresh)
     
    
    

#     gray= erosion

#     show('image1',gray)
    
    _,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)

    kernel = np.ones((2, 2), np.uint8)

    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    thresh = ~thresh
    
    x=gray.shape[0]
    y=gray.shape[1]
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(y-20,x-20))
    kernel = np.full((x-20,y-20),255, dtype="uint8")
    kernel = cv2.copyMakeBorder(kernel,10,10,10,10,cv2.BORDER_CONSTANT,0)
    
    thresh = cv2.bitwise_and(thresh, kernel, mask = None) 
    
    show('image1',thresh)
    
    return gray, thresh


# Blackhat
# Threshold
# Dilate
# Erode
# Erode

In [16]:
def extract_num(gray, thresh):
    (_,cnts,_) = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

#     cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0] + cv2.boundingRect(x)[1] * gray.shape[1] )
#     x + y * w

    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])


    avgCntArea = np.mean([cv2.contourArea(k) for k in cnts])
    numbers = []
    rects = []
    
    for (i,c) in enumerate(cnts):
        mask = np.zeros(gray.shape,dtype="uint8")
        (x,y,w,h) = cv2.boundingRect(c)
        hull = cv2.convexHull(c)
        cv2.drawContours(mask,[hull],-1,255,-1)
        mask = cv2.bitwise_and(thresh,thresh,mask=mask)
        digit = mask[y-8:y+h+8,x-8:x+w+8]
        
        if (h/w>=15 or w/h>=15 or w*h<30):
            continue
            
        if digit.shape[0]!=0 and digit.shape[1]!=0:
#             show('digit',digit)
    #         kernel = np.ones((1, 1), np.uint8)
    #         thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#             show('digit',digit)
            digit = cv2.resize(digit,(28,28))
#             _,digit = cv2.threshold(digit,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
#             show('digit',digit)
    #         kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1,1))
    #         digit = cv2.erode(digit,kernel,iterations = 3)
            show('digit',digit)
            rects.append((x,y,w,h))
            numbers.append(digit)
            
    return rects, numbers

In [23]:
def predict(cell_name):
    
    path = "/home/akshay/SEM6/IVP/cell extraction only/cells_sample5/"
#     path = "/home/darshan/IVP material/mini project/cells/"
#     path = ''
    str_num = ''
    one = 1
    decimal_count = 0
    
    image = cv2.imread(path + cell_name)
    
    gray, thresh = pre_processing(image)
          
    rects, numbers = extract_num(gray, thresh)
    
    image = imutils.resize(image,width=150)
    
    size=len(numbers)
    
    for i in range(len(numbers)):
        (x,y,w,h) = rects[i]
        if (h/w>=15 or w/h>=15):
            continue
        str_num, decimal_count, d_flag, image = decimal_check(decimal_count,i,rects,size,image,str_num)
        one_flag = one_check(i,rects)
        if d_flag:
            continue
        elif one_flag:
            num = one
            str_num += str(one)
        else:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
            digit = cv2.erode(numbers[i],kernel,iterations = 1)
            digit = digit.reshape(1,28,28,1)
            prediction = new_model.predict(digit)
            num = np.argmax(prediction)
            str_num += str(num)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
        cv2.putText(image,str(num),(x+2,y-5),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,255,0),2,cv2.LINE_AA)
    
    show('image',image)
    return str_num
    

In [24]:
def show(title,image):
#     cv2.imshow(title,image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return

In [25]:
import numpy as np
import cv2
import imutils
import tensorflow as T

new_model=T.keras.models.load_model('/home/akshay/SEM6/IVP/Project/new_model.model')
# new_model=T.keras.models.load_model('mnist_model_new.h5')

super_l = []
l = []
str_num = ''

start_cell = 0
end_cell = 5
 
for i in range(0, 72):
    cell_name = str(i) + ".jpg"
    str_num = predict(cell_name)
    l.append(str_num + '---' + str(i))
    str_num = ''
#     if (i+1) % (end_cell - start_cell + 1) == 0:
#         super_l.append(l)
#         l=[]
        
print(super_l)
print(l)

[]
['05---0', '---1', '---2', '---3', '---4', '05---5', '04---6', '04---7', '---8', '---9', '---10', '08---11', '1.5---12', '04---13', '---14', '---15', '---16', '5.5---17', '04---18', '05---19', '---20', '---21', '---22', '09---23', '02---24', '01---25', '---26', '---27', '---28', '03---29', '---30', '---31', '---32', '---33', '---34', '---35', '---36', '---37', '---38', '---39', '---40', '---41', '---42', '---43', '---44', '---45', '---46', '---47', '---48', '---49', '---50', '---51', '---52', '---53', '---54', '---55', '---56', '---57', '---58', '---59', '---60', '---61', '---62', '70221---63', '8243---64', '30.5---65', '800.0---66', '---67', '---68', '---69', '---70', '---71']
